In [ ]:
from Models.model_sources.markov_source import MarkovChain
from Models.MMC import MMC
from Models.DBN import FMC
import requests
from Models.model_sources.mtd_source import MTD
from Models.model_sources.path_encoder import PathEncoder, SequenceCutter
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2

In [26]:
# IDeas here:
# Use indicators
#

In [27]:
interval = "5m"
symbol = "AAPL.US"
def get_crypto_price(symbol, exchange,interval, start_date = None):
    api_key = '630d712fd8bd74.01254339'
    api_url = f'https://eodhistoricaldata.com/api/intraday/{symbol}?api_token={api_key}&interval={interval}&fmt=json'
    raw_df = requests.get(api_url).json()
    df = pd.DataFrame(raw_df)
    df['close'] = df['close'].astype(float)
    return df

# go for 100 states
# add volume, increase to 24 states
# fix training data for MTD
#


df = get_crypto_price(symbol = symbol, exchange = 'USD', interval = interval)
print(df.head())
df = df.drop(["gmtoffset", "datetime", "timestamp","volume"], axis=1)

df['Change'] = df['close'].diff()

df['Change_enc'] = np.nan
std = df.std()['Change']

df.loc[df.Change < 0.0, 'Change_enc'] = "drop"

df.loc[df.Change <= -std, 'Change_enc'] = "big drop"
df.loc[df.Change <= -2*std, 'Change_enc'] = "bigger drop"
df.loc[df.Change <= -3*std, 'Change_enc'] = "biggest drop"


df.loc[df.Change > 0.0, 'Change_enc'] = "rise"

df.loc[df.Change >= std, 'Change_enc'] = "big rise"
df.loc[df.Change >= 2*std, 'Change_enc'] = "bigger rise"
df.loc[df.Change >= 3*std, 'Change_enc'] = "biggest rise"



# New York
# London
# Asia

#df['volume_bin'] = pd.qcut(df['volume'], q=6, labels=[i for i in range(6)])
df['open_bin'] = pd.qcut(df['open'], q=4, labels=[i for i in range(4)])
df['close_bin'] = pd.qcut(df['close'], q=4, labels=[i for i in range(4)])

df['ma5'] = df['Change'].rolling(window=5, min_periods=5).mean()

df['ma8'] = df['Change'].rolling(window=8, min_periods=8).mean()
df['ma13'] = df['Change'].rolling(window=13, min_periods=13).mean()



columns = ["close_bin","open_bin"] #"volume_bin"]

for c in columns:
    res = [None, None, None]
    res.extend([(df.iloc[i-1][c],df.iloc[i-2][c], df.iloc[i-3][c])  for i in range(3, len(df))])
    df[c+'_history'] = res

#df.dropna(inplace=True)
df = df.reset_index()

df['ma_cross'] = [[x for _,x in sorted(zip([row['ma5'],row['ma8'],row['ma13']],[0,1,2]))] for key,row in df.iterrows()]


df['gain'] = df['Change'].clip(lower=0).round(2)
df['loss'] = df['Change'].clip(upper=0).abs().round(2)
df['gain'] = df['gain'].fillna(0)
df['loss'] = df['loss'].fillna(0)


window_length = 14
df['avg_gain'] = df['gain'].rolling(window=window_length, min_periods=window_length).mean()
df['avg_loss'] = df['loss'].rolling(window=window_length, min_periods=window_length).mean()



    timestamp  gmtoffset             datetime        open        high  \
0  1653053400          0  2022-05-20 13:30:00  139.000000  139.580001   
1  1653053700          0  2022-05-20 13:35:00  138.660003  138.910003   
2  1653054000          0  2022-05-20 13:40:00  138.809997  139.929992   
3  1653054300          0  2022-05-20 13:45:00  139.789993  140.179992   
4  1653054600          0  2022-05-20 13:50:00  139.619995  140.359695   

          low       close     volume  
0  138.500000  138.651992  7433897.0  
1  138.070007  138.809997  2606685.0  
2  138.679992  139.785003  2955811.0  
3  139.161804  139.612304  2670813.0  
4  139.539993  140.239898  2352598.0  


In [28]:
df['ma_cross'] = [''.join([str(x) for _,x in sorted(zip([row['ma5'],row['ma8'],row['ma13']],[0,1,2]))]) for key,row in df.iterrows()]

In [29]:
df['Change'].rolling(window=5, min_periods=5).mean()

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
6394    0.023999
6395    0.023999
6396    0.149420
6397    0.134003
6398    0.116998
Name: Change, Length: 6399, dtype: float64

In [30]:
window_length = 14
# Get WMS averages
# Average Gains
for i, row in enumerate(df['avg_gain'].iloc[window_length+1:]):
    df['avg_gain'].iloc[i + window_length + 1] = \
        (df['avg_gain'].iloc[i + window_length] *
         (window_length - 1) +
         df['gain'].iloc[i + window_length + 1]) \
        / window_length
# Average Losses
for i, row in enumerate(df['avg_loss'].iloc[window_length+1:]):
    df['avg_loss'].iloc[i + window_length + 1] = \
        (df['avg_loss'].iloc[i + window_length] *
         (window_length - 1) +
         df['loss'].iloc[i + window_length + 1]) \
        / window_length
# View initial results
print(df[window_length-1:window_length+5])

/tmp/ipykernel_18187/3763601371.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['avg_gain'].iloc[i + window_length + 1] = \
/tmp/ipykernel_18187/3763601371.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['avg_loss'].iloc[i + window_length + 1] = \


    index        open        high         low       close    Change  \
13     13  138.589904  138.789993  138.270004  138.380004 -0.209992   
14     14  138.369995  138.800003  138.309997  138.520004  0.140000   
15     15  138.529998  138.639999  137.960006  138.009994 -0.510010   
16     16  138.000000  138.089996  137.300003  137.374206 -0.635788   
17     17  137.369995  137.570007  137.059997  137.190002 -0.184204   
18     18  137.199996  137.529998  137.089996  137.399993  0.209991   

   Change_enc open_bin close_bin       ma5       ma8      ma13  \
13       drop        0         0 -0.052338 -0.186249 -0.020922   
14       rise        0         0 -0.039999 -0.173750 -0.022307   
15       drop        0         0 -0.120001 -0.088751 -0.136539   
16       drop        0         0 -0.281158 -0.158436 -0.172161   
17       drop        0         0 -0.279999 -0.191250 -0.234607   
18       rise        0         0 -0.196002 -0.151251 -0.190000   

   close_bin_history open_bin_history m

In [31]:
# Calculate RS Values
df['rs'] = df['avg_gain'] / df['avg_loss']
df = df.iloc[30:]
def convert_rs(value):
    if value > .70:
        return 0
    if value < .30:
        return 1
    return 2

df['rs_state'] = [convert_rs(df['rs'].values[i]) for i in range(len(df))]

## Fit models

In [32]:
values = [df.Change_enc, df.ma_cross, df.rs_state]

data_to_fit = np.array([''.join([str(v.values[i]) for v in values]) for i in range(len(df))])
#data_to_fit = np.array([str(v3[i])+str(v2[i]) for i in range(len(df))])
values = df.Change_enc.values

order = 2

sc = SequenceCutter(order)
x, y = sc.transform(data_to_fit)

pe = PathEncoder(order)
pe.fit(x, y)

x_tr, y_tr = pe.transform(x, y)


In [ ]:
from sklearn.model_selection import train_test_split

div = round(len(x_tr)*.85)
X_train, X_test, y_train, y_test = x_tr[div:, :],x_tr[:div, :], y_tr[div:], y_tr[:div]
(X_train, X_test, y_train, y_test) = train_test_split(x_tr, y_tr)

#X_train, y_train = x_tr, y_tr
#X_test, y_test = x_tr, y_tr

print(max(X_train.flatten()), len(set(X_train.flatten())))
amount_to_average = 1

sgo_type = "greedy"
methods = [MMC, FMC] #HMC] #FMC]
types = [m.__name__ for m in methods]

state_count = len(np.unique(x_tr))

mmc = None
for _ in range(amount_to_average):

    args_training = {"X_train": X_train, "y_train": y_train}
    args_testing =   {"X_test": X_test, "y_test": y_test}
    results_training = []
    results_testing = []

    for m in methods:
        model = m(state_count, order=order)
        training = MarkovChain.calculate_time(model.train, args_training)
        testing = MarkovChain.calculate_time(model.test, args_testing)

        if m.__name__ == "MMC":
            mmc = model
            pred_res = []
            drop_indexes = [key for key,value in pe.label_dict_inverse.items() if "drop" in value]
            rise_indexes = [key for key,value in pe.label_dict_inverse.items() if "drop" in value]

            for lag in X_test:


                prob_values = mmc.cpt[mmc.find_high(lag, mmc.index_dict)]

                drop_prob = sum([prob_values[key] for key in drop_indexes])

                if drop_prob > .5:
                    pred_res.append(drop_indexes[0])
                else:
                    pred_res.append(rise_indexes[0])
        else:
            pred_res = model.predict(X_test)

        #results_training.append(training[0])
        results_testing.append(testing[0])

        count = 0
        for i,p in enumerate(pred_res):

            if "drop" in pe.label_dict_inverse[p] and "drop" in pe.label_dict_inverse[y_test[i]]:
                count += 1
            elif "rise" in pe.label_dict_inverse[p] and "rise" in pe.label_dict_inverse[y_test[i]]:
                count += 1
        print(count/len(pred_res))


        #print(count/sum)

        print(model.__class__.__name__)
        print(f"Training: {training}")
        print(f"Testing: {testing}")
        print("")

    print(results_testing)

47 48


In [33]:
lag = X_test[0]



In [34]:
drop_prob

0.5428490351872871

In [ ]:
def create_bar_graph(data, title, types):
    plt.figure(figsize=(10, 5))

    # creating the bar plot
    plt.bar(types, data, color='maroon',
            width=0.4)

    plt.title(title)
    plt.show()
[m.__name__ for m in methods if m != MTD]

create_bar_graph(results_testing, f"{symbol} Intraday {interval} Interval Latest 15% Order {order}", [m.__name__ for m in methods if m != MTD])

In [37]:
[m.__name__ for m in methods]

['MMC', 'FMC', 'HMC']

## Choose model

In [39]:
mmc.predict([x_tr[-1]])

[0]